In [1]:
import dill
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [2]:
with open('../models/20210524_211225/FS_obj.p', 'rb') as f:
    fsobj = dill.load(f)
    
with open('../models/20210524_211225/ML_obj.p', 'rb') as f:
    mlobj = dill.load(f)


C:\Users\bioen\.conda\envs\brain_torch\lib\site-packages\sklearn\base.py:315: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.23.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\bioen\.conda\envs\brain_torch\lib\site-packages\sklearn\base.py:315: UserWarning: Trying to unpickle estimator RFECV from version 0.23.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\bioen\.conda\envs\brain_torch\lib\site-packages\sklearn\base.py:315: UserWarning: Trying to unpickle estimator LinearSVC from version 0.23.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\bioen\.conda\envs\brain_torch\lib\site-packages\sklearn\base.py:315: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.23.2 when using version 0.24.2. This

In [3]:
df = pd.read_csv('../models/20210524_211225/group_df_afterFixation.csv', index_col='subj_id')
df.columns

Index(['area_lbankssts_medM2080', 'area_lbankssts_medP2080',
       'area_lcaudalanteriorcingulate_medM2080',
       'area_lcaudalanteriorcingulate_medP2080',
       'area_lcaudalmiddlefrontal_medM2080',
       'area_lcaudalmiddlefrontal_medP2080', 'area_lcuneus_medM2080',
       'area_lcuneus_medP2080', 'area_lentorhinal_medM2080',
       'area_lentorhinal_medP2080',
       ...
       'volume_rtemporalpole_medM2080', 'volume_rtemporalpole_medP2080',
       'volume_rtransversetemporal_medM2080',
       'volume_rtransversetemporal_medP2080', 'DX_GROUP', 'AGE_AT_SCAN ',
       'SEX', 'SRS_COMMUNICATION_T', 'categories_COMMUNICATION', 'my_labels'],
      dtype='object', length=550)

In [4]:
df.drop(['SEX','AGE_AT_SCAN ', 'DX_GROUP', 'SRS_COMMUNICATION_T','categories_COMMUNICATION'], axis=1, inplace=True)
df.rename({'my_labels':'DX_GROUP'}, axis=1, inplace=True)

In [5]:
y = df['DX_GROUP']
Xs = MinMaxScaler().fit_transform(df.drop('DX_GROUP', axis=1))

In [6]:
Xselected = {}
for key, fs in fsobj.items():
    Xselected[key] = Xs[:, np.where(fs.support_)[0]]

In [10]:
for fkey, Xsele in Xselected.items():
    for mkey, ml in mlobj[fkey].items():
        Xtrain, Xtest, ytrain, ytest = train_test_split(Xsele, y, test_size=0.1, random_state=114, shuffle=True, stratify=y.values)
        est = ml.best_estimator_
        est.fit(Xtrain, ytrain)
        yhat = est.predict(Xtest)
        print(f'Features selected with {fkey} and ML model is {mkey}')
        print(confusion_matrix(ytest, yhat))

Features selected with lr and ML model is nn
[[ 0 17]
 [ 0 17]]
Features selected with lsvm and ML model is nn
[[17  0]
 [17  0]]
Features selected with xgb and ML model is nn
[[12  5]
 [ 8  9]]
Features selected with rf and ML model is nn
[[ 4 13]
 [ 6 11]]
Features selected with gradboost and ML model is nn
[[ 7 10]
 [ 5 12]]


In [10]:
y.values.shape

(332,)

In [11]:
Xsele.shape

(332, 34)